This demo is from demo 5 in [1].

In [1]:
from sympy import symbols, Matrix
from SOSPy import *
import numpy as np
import math

## Upper  Bound of Strucutred Singular Value


According to Proposition 8.25 of [1] and Theorem 6.1 of [2], we can derive the following conditions. Given a matrix $M \in \mathbb{C}^{n \times n}$ and structed scalar uncertanties $ \Delta = \text{diag}(\delta_1, \delta_2, \dots, \delta_n)$, $\delta_i \in \mathbb{C}$, the structured singular value $\mu(M,\Delta)$ is less than $\gamma$, if there exists solutions $Q_i \geq 0 \in \mathbb{R}^{2n \times 2n}, T_i \in \mathbb{R}^{2n \times 2n}$ and $r_{ij} \geq 0$ such that 

\begin{gather*}
    -\sum_{i=1}^n \, Q_i(x) A_i(x) - \sum_{1\leq i < j \leq n} \, r_{ij} A_i(x) A_j(x) + I(x) \geq 0 \tag{1}
\end{gather*}

where $x\in \mathbb{R}^{2n}$,

\begin{gather}
    Q_i(x) = x^TQ_ix,\\
    I(x) = -\sum^{2n}_{i=1} \, x_i^2, \\
    A_i(x) = x^TA_ix, \\
    A_i = \begin{bmatrix} Re(H_i) & -Im(H_i) \\ Im(H_i) & Re(H_i) \end{bmatrix}, \\
    H_i = M^*e_i^*e_iM - \gamma^2 e_i^*e_i,
\end{gather}

and $e_i$ is the $i$-th unit vector in $\mathbb{C}^n$.

Thus the SOSP for this problem can be formulated as follows:

Choose a fixed value of $\gamma$. For $I(x)$ and $A_i(x)$ as described above, find sums of squares

\begin{align*}
    Q_i(x) &= x^TQ_ix, \quad \text{ for } i = 1, \dots, 2n, \\
    r_{ij} &\geq 0  \text{  (zero order SOS), for } 1 \leq i < j \leq 2n,
\end{align*}

such that (1) is satisfied.

The optimal value of $\gamma$ can be found for example by bisection. In this example, we consider the following $M$:

\begin{gather}
    M = UV^*, \quad U = \begin{bmatrix} a & 0 \\ b & b \\ c & jc \\ d & f \end{bmatrix}, \quad
    V = \begin{bmatrix} 0&a\\b&-b\\c&-jc\\-jf&-d \end{bmatrix}
\end{gather}

with $a = \sqrt{\frac{2}{\alpha}},b = c = \frac{1}{\sqrt{\alpha}},d=-\sqrt{\frac{\beta}{\alpha}},f = (1+j)\sqrt{\frac{1}{(\beta\alpha)}},\alpha = 3+\sqrt{3},\beta=\sqrt{3}-1$. It is know that $\mu(M,\Delta) \approx 0.8723$. In this example, we can prove that $\mu(M,\Delta) < 0.8724$.

In [2]:
# Running this code may take a while
x0,x1,x2,x3,x4,x5,x6,x7 = symbols("x0,x1,x2,x3,x4,x5,x6,x7")
vartable = [x0,x1,x2,x3,x4,x5,x6,x7]

# The matrix under consideration
alpha = 3 + math.sqrt(3)
beta = math.sqrt(3) - 1
a = math.sqrt(2/alpha)
b = 1/math.sqrt(alpha)
c = b
d = -math.sqrt(beta/alpha)
f = (1 + 1j)*math.sqrt(1/(alpha*beta))
U = np.array([[a,0],[b,b],[c,1j*c],[d,f]], dtype=complex)
V = np.array([[0,a],[b,-b],[c,-1j*c],[-1j*f,-d]], dtype=complex)
M = np.dot(U, V.conj().T)

# Constructing A(x)'s
gam = 0.8724

Z = monomials(vartable,[1])
Z_mat = Matrix(Z)

A = []
for i in range(4):
    M_row = M[i,:]
    H = np.dot(M_row[None,:].conj().T,M_row[None,:])
    H[i,i] = H[i,i] - gam**2
    H_real = H.real
    H_imag = H.imag
    H_block = np.vstack((np.hstack((H_real,-H_imag)),np.hstack((H_imag,H_real))))
    H_mat = Matrix(H_block.tolist())
    A.append((Z_mat.transpose()*H_mat*Z_mat)[0])

    
# =============================================
# Initialize the sum of squares program
prog = sosprogram(vartable)

# =============================================
# Define SOSP variables

# -- Q(x)'s -- : sums of squares
# Monomial vector: [x1; ... x8]
Q = []
for i in range(4):
    prog,V = sossosvar(prog,Z)
    Q.append(V)

# -- r's -- : constant sum of squares
Z = monomials(vartable,[0])
r = Matrix.zeros(4)
for i in range(4):
    for j in range(i+1,4):
        prog,r[i,j] = sossosvar(prog,Z)
        
# =============================================
# Next, define SOSP constraints

# Constraint : -sum(Qi(x)*Ai(x)) - sum(rij*Ai(x)*Aj(x)) + I(x) >= 0
expr = 0
# Adding term
for i in range(4):
    expr = expr - A[i]*Q[i]
    
for i in range(4):
    for j in range(i+1,4):
        expr = expr - A[i]*A[j]*r[i,j]
        
# Constant term: I(x) = -(x1^4 + ... + x8^4)
v_4 = [item**4 for item in vartable]
I = -np.sum(v_4)
expr = expr + I

prog = sosineq(prog,expr)

options = {}
options['solver'] = 'cvxopt'
prog = sossolve(prog,options,verbose=0)

Installed SDP solvers:  ['MOSEK', 'CVXOPT', 'SCS', 'SDPA']

 Residual norm 9.74196406409413e-11
cpusec: 4.01576
iter: 13
status: optimal
pinf: 0.0
dinf: 0.0


The program is feasible, so 0.8724 is an upper bound for $\mu$.

### Citation:

[1]: A. Papachristodoulou, J. Anderson, G. Valmorbida, S. Prajna, P. Seiler, P. A. Parrilo, M. M. Peet, and D. Jagt, "4.5 Upper Bound of Structured Singular Value," in _Sum of Squares Optimization Toolbox for MATLAB, User’s guide_, Version 4.00, 2021, pp. 41-45.